# 预测波士顿房价

## 使用 SageMaker 更新模型

_机器学习工程师纳米学位课程 | 部署_

---

在此 notebook 中，我们将继续使用[波士顿房价数据集](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html)。我们的目标是训练两个不同的模型，并使用 SageMaker 让部署的端点切换使用两个模型中的某个模型。使用 SageMaker 完成这些操作的优势之一是我们可以更新模型，但是不中断服务。我们可以继续向端点发送数据，并且端点始终不会消失。

## 一般步骤

通常，在 notebook 实例中使用 SageMaker 时，你需要完成以下步骤。当然，并非每个项目都要完成每一步。此外，有很多步骤有很大的变化余地，你将在这些课程中发现这一点。

1. 下载或检索数据。
2. 处理/准备数据。
3. 将处理的数据上传到 S3。
4. 训练所选的模型。
5. 测试训练的模型（通常使用批转换作业）。
6. 部署训练的模型。
7. 使用部署的模型。

在此 notebook 中，我们将跳过第 5 步 - 测试模型。此外，我们将用不同的模型多次执行第 4、6 和 7 步。

## 第 0 步：设置 notebook

先进行必要的设置以运行 notebook。首先，加载所需的所有 Python 模块。

In [1]:
%matplotlib inline

import os

import numpy as np
import pandas as pd

from pprint import pprint
import matplotlib.pyplot as plt
from time import gmtime, strftime

from sklearn.datasets import load_boston
import sklearn.model_selection

除了上面的模块之外，我们还需要导入将使用的各种 SageMaker 模块。

In [2]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.predictor import csv_serializer

# This is an object that represents the SageMaker session that we are currently operating in. This
# object contains some useful information that we will need to access later such as our region.
session = sagemaker.Session()

# This is an object that represents the IAM role that we are currently assigned. When we construct
# and launch the training job later we will need to tell it what IAM role it should have. Since our
# use case is relatively simple we will simply assign the training job the role we currently have.
role = get_execution_role()

## 第 1 步：下载数据

幸运的是，我们可以使用 sklearn 检索数据集，所以这一步相对比较简单。

In [3]:
boston = load_boston()

## 第 2 步：准备和拆分数据

因为使用的是整洁的表格数据，所以不需要进行任何处理。但是，我们需要将数据集中的各行拆分成训练集、测试集和验证集。

In [4]:
# First we package up the input data and the target variable (the median value) as pandas dataframes. This
# will make saving the data to a file a little easier later on.

X_bos_pd = pd.DataFrame(boston.data, columns=boston.feature_names)
Y_bos_pd = pd.DataFrame(boston.target)

# We split the dataset into 2/3 training and 1/3 testing sets.
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X_bos_pd, Y_bos_pd, test_size=0.33)

# Then we split the training set further into 2/3 training and 1/3 validation sets.
X_train, X_val, Y_train, Y_val = sklearn.model_selection.train_test_split(X_train, Y_train, test_size=0.33)

## 第 3 步：将训练和验证文件上传到 S3

使用 SageMaker 创建训练作业后，进行训练操作的容器会执行。此容器可以访问存储在 S3 上的数据。所以我们需要将用来训练的数据上传到 S3。我们可以使用 SageMaker API 完成这一步，它会在后台自动处理一些步骤。

### 将数据保存到本地

首先，我们需要创建训练和验证 csv 文件，并将这些文件上传到 S3。

In [5]:
# This is our local data directory. We need to make sure that it exists.
data_dir = '../data/boston'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [6]:
# We use pandas to save our train and validation data to csv files. Note that we make sure not to include header
# information or an index as this is required by the built in algorithms provided by Amazon. Also, it is assumed
# that the first entry in each row is the target variable.

pd.concat([Y_val, X_val], axis=1).to_csv(os.path.join(data_dir, 'validation.csv'), header=False, index=False)
pd.concat([Y_train, X_train], axis=1).to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

### 上传到 S3

因为目前正在 SageMaker 会话中运行，所以可以使用代表此会话的对象将数据上传到默认的 S3 存储桶中。注意，建议提供自定义 prefix（即 S3 文件夹），以防意外地破坏了其他 notebook 或项目上传的数据。

In [7]:
prefix = 'boston-update-endpoints'

val_location = session.upload_data(os.path.join(data_dir, 'validation.csv'), key_prefix=prefix)
train_location = session.upload_data(os.path.join(data_dir, 'train.csv'), key_prefix=prefix)

## 第 4 步：训练 XGBoost 模型

将训练和验证数据上传到 S3 后，我们可以构建 XGBoost 模型并训练它。我们将使用高阶 SageMaker API 完成这一步，这样的话代码更容易读懂，但是灵活性较差。

为了构建一个评估器（即我们要训练的对象），我们需要提供训练代码所在的容器的位置。因为我们使用的是内置算法，所以这个容器由 Amazon 提供。但是，容器的完整名称比较长，取决于我们运行所在的区域。幸运的是，SageMaker 提供了一个实用方法，叫做 `get_image_uri`，它可以为我们构建镜像名称。

为了使用 `get_image_uri` 方法，我们需要向其提供当前所在区域（可以从 session 对象中获得），以及要使用的算法的名称。在此 notebook 中，我们将使用 XGBoost，但是你也可以尝试其他算法。[常见参数](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html)中列出了 Amazon 的内置算法。

In [8]:
# As stated above, we use this utility method to construct the image name for the training container.
xgb_container = get_image_uri(session.boto_region_name, 'xgboost')

# Now that we know which container to use, we can construct the estimator object.
xgb = sagemaker.estimator.Estimator(xgb_container, # The name of the training container
                                    role,      # The IAM role to use (our current role in this case)
                                    train_instance_count=1, # The number of instances to use for training
                                    train_instance_type='ml.m4.xlarge', # The type of instance ot use for training
                                    output_path='s3://{}/{}/output'.format(session.default_bucket(), prefix),
                                                                        # Where to save the output (the model artifacts)
                                    sagemaker_session=session) # The current SageMaker session

	get_image_uri(region, 'xgboost', '0.90-1').


在要求 SageMaker 开始训练作业之前，我们需要设置模型超参数。如果使用 XGBoost 算法，可以设置的超参数有很多，以下只是其中几个。如果你想修改下面的超参数或修改其他超参数，请参阅 [XGBoost 超参数页面](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html)

In [9]:
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        objective='reg:linear',
                        early_stopping_rounds=10,
                        num_round=200)

完全设置好 estimator 对象后，可以训练它了。我们需要告诉 SageMaker 输入数据是 csv 格式，然后调用 `fit` 方法。

In [10]:
# This is a wrapper around the location of our train and validation data, to make sure that SageMaker
# knows our data is in csv format.
s3_input_train = sagemaker.s3_input(s3_data=train_location, content_type='text/csv')
s3_input_validation = sagemaker.s3_input(s3_data=val_location, content_type='text/csv')

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

2020-03-31 11:00:10 Starting - Starting the training job...
2020-03-31 11:00:12 Starting - Launching requested ML instances......
2020-03-31 11:01:14 Starting - Preparing the instances for training...
2020-03-31 11:02:06 Downloading - Downloading input data...
2020-03-31 11:02:38 Training - Downloading the training image...
2020-03-31 11:03:10 Uploading - Uploading generated training model
2020-03-31 11:03:10 Completed - Training job completed
Arguments: train
[2020-03-31:11:02:58:INFO] Running standalone xgboost training.
[2020-03-31:11:02:58:INFO] File size need to be processed in the node: 0.02mb. Available memory size in the node: 8511.29mb
[2020-03-31:11:02:58:INFO] Determined delimiter of CSV input is ','
[11:02:58] S3DistributionType set as FullyReplicated
[11:02:58] 227x13 matrix with 2951 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2020-03-31:11:02:58:INFO] Determined delimiter of CSV input is ','
[11:02:58] S3DistributionType set as Ful

Training seconds: 64
Billable seconds: 64


## 第 5 步：测试训练过的模型

我们暂时将跳过这一步。


## 第 6 (A) 步：部署训练过的模型

虽然我们使用了高阶方法构建和训练 XGBoost 模型，但是我们将使用低阶方法部署模型。原因之一是可以更好地控制端点的构建方式。在后面构建更高级的端点时你就会明白的。

### 构建模型

在部署模型之前，我们需要先创建模型。我们之前使用的 `fit` 方法创建了一些模型工件，我们可以使用这些工件构建模型对象。

In [11]:
# Remember that a model needs to have a unique name
xgb_model_name = "boston-update-xgboost-model" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

# We also need to tell SageMaker which container should be used for inference and where it should
# retrieve the model artifacts from. In our case, the xgboost container that we used for training
# can also be used for inference and the model artifacts come from the previous call to fit.
xgb_primary_container = {
    "Image": xgb_container,
    "ModelDataUrl": xgb.model_data
}

# And lastly we construct the SageMaker model
xgb_model_info = session.sagemaker_client.create_model(
                                ModelName = xgb_model_name,
                                ExecutionRoleArn = role,
                                PrimaryContainer = xgb_primary_container)

### 创建端点配置

创建了模型之后，我们可以创建端点了。我们需要首先创建端点配置，实际上就是 SageMaker 用来构建端点本身所使用的蓝图。

In [12]:
# As before, we need to give our endpoint configuration a name which should be unique
xgb_endpoint_config_name = "boston-update-xgboost-endpoint-config-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

# And then we ask SageMaker to construct the endpoint configuration
xgb_endpoint_config_info = session.sagemaker_client.create_endpoint_config(
                            EndpointConfigName = xgb_endpoint_config_name,
                            ProductionVariants = [{
                                "InstanceType": "ml.m4.xlarge",
                                "InitialVariantWeight": 1,
                                "InitialInstanceCount": 1,
                                "ModelName": xgb_model_name,
                                "VariantName": "XGB-Model"
                            }])

### 部署端点

创建了端点配置后，我们可以要求 SageMaker 构建端点了。

**注意：**再次友好提醒你，你即将部署端点。使用完毕后，请记得关闭端点！

In [13]:
# Again, we need a unique name for our endpoint
endpoint_name = "boston-update-endpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

# And then we can deploy our endpoint
endpoint_info = session.sagemaker_client.create_endpoint(
                    EndpointName = endpoint_name,
                    EndpointConfigName = xgb_endpoint_config_name)

In [14]:
endpoint_dec = session.wait_for_endpoint(endpoint_name)

-------------!

## 第 7 (A) 步：使用模型

训练并部署模型后，我们可以向其发送一些测试数据，看看结果如何。

In [15]:
response = session.sagemaker_runtime_client.invoke_endpoint(
                                                EndpointName = endpoint_name,
                                                ContentType = 'text/csv',
                                                Body = ','.join(map(str, X_test.values[0])))

In [16]:
pprint(response)

{'Body': <botocore.response.StreamingBody object at 0x7f7e0d3a25f8>,
 'ContentType': 'text/csv; charset=utf-8',
 'InvokedProductionVariant': 'XGB-Model',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '13',
                                      'content-type': 'text/csv; charset=utf-8',
                                      'date': 'Tue, 31 Mar 2020 11:12:21 GMT',
                                      'x-amzn-invoked-production-variant': 'XGB-Model',
                                      'x-amzn-requestid': '4e0acaf7-3c84-4490-b92a-ef317e907bbd'},
                      'HTTPStatusCode': 200,
                      'RequestId': '4e0acaf7-3c84-4490-b92a-ef317e907bbd',
                      'RetryAttempts': 0}}


In [17]:
result = response['Body'].read().decode("utf-8")

In [18]:
pprint(result)

'25.4541683197'


In [19]:
Y_test.values[0]

array([24.6])

## 关闭端点

现在我们知道 XGBoost 端点能运行，我们将关闭端点。稍后我们将再使用它。

In [20]:
session.sagemaker_client.delete_endpoint(EndpointName = endpoint_name)

{'ResponseMetadata': {'RequestId': '7b2cf3dd-d894-4fa4-ad24-2512c94ffcd4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7b2cf3dd-d894-4fa4-ad24-2512c94ffcd4',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Tue, 31 Mar 2020 11:12:29 GMT'},
  'RetryAttempts': 0}}

## 第 4 (B) 步：训练线性模型

假设对于我们所在的环境，我们之前训练的 XGBoost 模型成本变得非常高。也许对端点的调用次数增加了，使用 XGBoost 模型进行推理所需的时间变得很长。

一种可能的解决方法是训练一个更简单的模型，看看它能否几乎达到一样的性能。我们将构建一个线性模型。构建流程与之前创建的 XGBoost 模型一样，但是我们需要设置不同的超参数。

In [21]:
# Similar to the XGBoost model, we will use the utility method to construct the image name for the training container.
linear_container = get_image_uri(session.boto_region_name, 'linear-learner')

# Now that we know which container to use, we can construct the estimator object.
linear = sagemaker.estimator.Estimator(linear_container, # The name of the training container
                                        role,      # The IAM role to use (our current role in this case)
                                        train_instance_count=1, # The number of instances to use for training
                                        train_instance_type='ml.m4.xlarge', # The type of instance ot use for training
                                        output_path='s3://{}/{}/output'.format(session.default_bucket(), prefix),
                                                                            # Where to save the output (the model artifacts)
                                        sagemaker_session=session) # The current SageMaker session

在要求 SageMaker 训练模型之前，我们需要设置一些超参数。因为我们使用的是线性模型，所以要设置的超参数少了很多。详情请参阅[线性模型超参数](https://docs.aws.amazon.com/sagemaker/latest/dg/ll_hyperparameters.html)页面

In [22]:
linear.set_hyperparameters(feature_dim=13, # Our data has 13 feature columns
                           predictor_type='regressor', # We wish to create a regression model
                           mini_batch_size=200) # Here we set how many samples to look at in each iteration

设置好超参数后，我们可以要求 SageMaker 使线性模型拟合我们的数据。

In [23]:
linear.fit({'train': s3_input_train, 'validation': s3_input_validation})

2020-03-31 11:12:39 Starting - Starting the training job...
2020-03-31 11:12:40 Starting - Launching requested ML instances...
2020-03-31 11:13:39 Starting - Preparing the instances for training.........
2020-03-31 11:15:02 Downloading - Downloading input data...
2020-03-31 11:15:40 Training - Training image download completed. Training in progress.
2020-03-31 11:15:40 Uploading - Uploading generated training model.Docker entrypoint called with argument(s): train
Running default environment configuration script
[03/31/2020 11:15:37 INFO 140708599981888] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'feature_dim': u'auto', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5


2020-03-31 11:15:48 Completed - Training job completed
Training seconds: 46
Billable seconds: 46


## 第 6 (B) 步：部署训练过的模型

与 XGBoost 模型相似，拟合模型后，我们需要部署模型。并且与 XGBoost 模型一样，我们将使用低阶方法，这样可以更好地控制端点创建过程。

### 构建模型
在部署模型之前，我们需要先创建模型。我们之前使用的 `fit` 方法创建了一些模型工件，我们可以使用这些工件构建模型对象。

In [24]:
# First, we create a unique model name
linear_model_name = "boston-update-linear-model" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

# We also need to tell SageMaker which container should be used for inference and where it should
# retrieve the model artifacts from. In our case, the linear-learner container that we used for training
# can also be used for inference.
linear_primary_container = {
    "Image": linear_container,
    "ModelDataUrl": linear.model_data
}

# And lastly we construct the SageMaker model
linear_model_info = session.sagemaker_client.create_model(
                                ModelName = linear_model_name,
                                ExecutionRoleArn = role,
                                PrimaryContainer = linear_primary_container)

### 创建端点配置

创建了模型之后，我们将创建端点配置，从而创建端点。

In [25]:
# As before, we need to give our endpoint configuration a name which should be unique
linear_endpoint_config_name = "boston-linear-endpoint-config-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

# And then we ask SageMaker to construct the endpoint configuration
linear_endpoint_config_info = session.sagemaker_client.create_endpoint_config(
                            EndpointConfigName = linear_endpoint_config_name,
                            ProductionVariants = [{
                                "InstanceType": "ml.m4.xlarge",
                                "InitialVariantWeight": 1,
                                "InitialInstanceCount": 1,
                                "ModelName": linear_model_name,
                                "VariantName": "Linear-Model"
                            }])

### 部署端点

创建了端点配置后，我们可以要求 SageMaker 构建端点了。

**注意：**再次友好提醒你，你即将部署端点。使用完毕后，请记得关闭端点！

In [26]:
# Again, we need a unique name for our endpoint
endpoint_name = "boston-update-endpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

# And then we can deploy our endpoint
endpoint_info = session.sagemaker_client.create_endpoint(
                    EndpointName = endpoint_name,
                    EndpointConfigName = linear_endpoint_config_name)

In [27]:
endpoint_dec = session.wait_for_endpoint(endpoint_name)

-----------!

## 第 7 (B) 步：使用模型

与 XGBoost 模型一样，我们将向端点发送一些数据，看看能否正常运行。需要注意的是，线性模型的输出格式与 XGBoost 模型的不同。

In [28]:
response = session.sagemaker_runtime_client.invoke_endpoint(
                                                EndpointName = endpoint_name,
                                                ContentType = 'text/csv',
                                                Body = ','.join(map(str, X_test.values[0])))

In [29]:
pprint(response)

{'Body': <botocore.response.StreamingBody object at 0x7f7e0d38ffd0>,
 'ContentType': 'application/json',
 'InvokedProductionVariant': 'Linear-Model',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '47',
                                      'content-type': 'application/json',
                                      'date': 'Tue, 31 Mar 2020 11:23:55 GMT',
                                      'x-amzn-invoked-production-variant': 'Linear-Model',
                                      'x-amzn-requestid': 'eb92cf38-f6a2-409e-8840-2948322f3306'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'eb92cf38-f6a2-409e-8840-2948322f3306',
                      'RetryAttempts': 0}}


In [30]:
result = response['Body'].read().decode("utf-8")

In [31]:
pprint(result)

'{"predictions": [{"score": 26.73609733581543}]}'


In [32]:
Y_test.values[0]

array([24.6])

## 关闭端点

现在我们知道线性模型的端点能运行，我们可以关闭端点了。

In [33]:
session.sagemaker_client.delete_endpoint(EndpointName = endpoint_name)

{'ResponseMetadata': {'RequestId': '01b4cb10-c4d3-43a4-9596-b07b02fc2a2b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '01b4cb10-c4d3-43a4-9596-b07b02fc2a2b',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Tue, 31 Mar 2020 11:24:00 GMT'},
  'RetryAttempts': 0}}

## 第 6 (C) 步：部署组合模型

到目前为止，我们构建了两个可以部署和使用的不同模型。在讲解如何使部署的端点从一个端点配置切换到另一个配置之前，先考虑一个稍微不同的情形。假设在从仅使用 XGBoost 切换到仅使用线性模型之前，我们首先想要执行 A/B 测试，将某些输入数据发送给 XGBoost 模型，将某些数据发送给线性模型。

幸运的是，SageMaker 提供了这样的功能。并且方法与部署模型差别不大。唯一区别是我们需要在端点配置的生产变体参数中列出多个模型。

有个问题值得思考，要向在生产变体参数中列出的每个模型发送多少数据？答案是取决于每个模型的权重。

假设在生产变体参数中列出了 $k$ 个模型，每个模型 $i$ 的权重为 $w_i$。那么每个模型 $i$ 将获得 $w_i / W$ 的数据，其中 $W = \sum_{i} w_i$。

因为我们有两个模型：线性模型和 XGBoost 模型，并且每个模型的权重为 1，所以每个模型将从发送给端点的数据中获得 1 / (1 + 1) = 1/2 的数据。

In [34]:
# As before, we need to give our endpoint configuration a name which should be unique
combined_endpoint_config_name = "boston-combined-endpoint-config-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

# And then we ask SageMaker to construct the endpoint configuration
combined_endpoint_config_info = session.sagemaker_client.create_endpoint_config(
                            EndpointConfigName = combined_endpoint_config_name,
                            ProductionVariants = [
                                { # First we include the linear model
                                    "InstanceType": "ml.m4.xlarge",
                                    "InitialVariantWeight": 1,
                                    "InitialInstanceCount": 1,
                                    "ModelName": linear_model_name,
                                    "VariantName": "Linear-Model"
                                }, { # And next we include the xgb model
                                    "InstanceType": "ml.m4.xlarge",
                                    "InitialVariantWeight": 1,
                                    "InitialInstanceCount": 1,
                                    "ModelName": xgb_model_name,
                                    "VariantName": "XGB-Model"
                                }])

创建了端点配置后，我们可以要求 SageMaker 构建端点了。

**注意：**再次友好提醒你，你即将部署端点。使用完毕后，请记得关闭端点！

In [35]:
# Again, we need a unique name for our endpoint
endpoint_name = "boston-update-endpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

# And then we can deploy our endpoint
endpoint_info = session.sagemaker_client.create_endpoint(
                    EndpointName = endpoint_name,
                    EndpointConfigName = combined_endpoint_config_name)

In [36]:
endpoint_dec = session.wait_for_endpoint(endpoint_name)

-------------!

## 第 7 (C) 步：使用模型

我们构建了向 XGBoost 模型和线性模型发送数据的端点，下面可以向端点发送一些数据，看看结果如何。

In [37]:
response = session.sagemaker_runtime_client.invoke_endpoint(
                                                EndpointName = endpoint_name,
                                                ContentType = 'text/csv',
                                                Body = ','.join(map(str, X_test.values[0])))
pprint(response)

{'Body': <botocore.response.StreamingBody object at 0x7f7e0e584780>,
 'ContentType': 'text/csv; charset=utf-8',
 'InvokedProductionVariant': 'XGB-Model',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '13',
                                      'content-type': 'text/csv; charset=utf-8',
                                      'date': 'Tue, 31 Mar 2020 11:31:04 GMT',
                                      'x-amzn-invoked-production-variant': 'XGB-Model',
                                      'x-amzn-requestid': '84bccf7b-8ace-4b62-ac4d-e40724074b58'},
                      'HTTPStatusCode': 200,
                      'RequestId': '84bccf7b-8ace-4b62-ac4d-e40724074b58',
                      'RetryAttempts': 0}}


因为查看一个响应结果并不足以了解运行状况，所以我们将查看几个不同的端点响应结果。

In [38]:
for rec in range(10):
    response = session.sagemaker_runtime_client.invoke_endpoint(
                                                EndpointName = endpoint_name,
                                                ContentType = 'text/csv',
                                                Body = ','.join(map(str, X_test.values[rec])))
    pprint(response)
    result = response['Body'].read().decode("utf-8")
    print(result)
    print(Y_test.values[rec])

{'Body': <botocore.response.StreamingBody object at 0x7f7e0e57a6a0>,
 'ContentType': 'application/json',
 'InvokedProductionVariant': 'Linear-Model',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '47',
                                      'content-type': 'application/json',
                                      'date': 'Tue, 31 Mar 2020 11:31:06 GMT',
                                      'x-amzn-invoked-production-variant': 'Linear-Model',
                                      'x-amzn-requestid': '76133065-7f94-4dfa-8a02-b592c13a46c5'},
                      'HTTPStatusCode': 200,
                      'RequestId': '76133065-7f94-4dfa-8a02-b592c13a46c5',
                      'RetryAttempts': 0}}
{"predictions": [{"score": 26.73609733581543}]}
[24.6]
{'Body': <botocore.response.StreamingBody object at 0x7f7e0e57a400>,
 'ContentType': 'application/json',
 'InvokedProductionVariant': 'Linear-Model',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '48',
               

如果我们不记得部署端点的属性后，可以使用 `describe_endpoint` 函数来要求 SageMaker 返回部署端点的描述信息。

In [39]:
pprint(session.sagemaker_client.describe_endpoint(EndpointName=endpoint_name))

{'CreationTime': datetime.datetime(2020, 3, 31, 11, 24, 4, 276000, tzinfo=tzlocal()),
 'EndpointArn': 'arn:aws:sagemaker:us-west-1:270372225889:endpoint/boston-update-endpoint-2020-03-31-11-24-04',
 'EndpointConfigName': 'boston-combined-endpoint-config-2020-03-31-11-24-02',
 'EndpointName': 'boston-update-endpoint-2020-03-31-11-24-04',
 'EndpointStatus': 'InService',
 'LastModifiedTime': datetime.datetime(2020, 3, 31, 11, 30, 12, 146000, tzinfo=tzlocal()),
 'ProductionVariants': [{'CurrentInstanceCount': 1,
                         'CurrentWeight': 1.0,
                         'DeployedImages': [{'ResolutionTime': datetime.datetime(2020, 3, 31, 11, 24, 5, 971000, tzinfo=tzlocal()),
                                             'ResolvedImage': '632365934929.dkr.ecr.us-west-1.amazonaws.com/linear-learner@sha256:1cc553330fc7ab939e72cc0c0ed4bed61bbb2e7b33b4f838cb0a146d0bb5da9c',
                                             'SpecifiedImage': '632365934929.dkr.ecr.us-west-1.amazonaws.com/l

## 更新端点

假设我们执行了 A-B 测试，并且新线性模型效果不错。现在我们将使端点从同时向 XGBoost 模型和线性模型发送数据切换到仅向线性模型发送数据。

但是我们不希望真正地关闭端点，因为这样会干扰依赖于端点的服务。我们可以要求 SageMaker 将端点**更新**成新的端点配置。

SageMaker 将使用新的配置设置新的端点。新端点运行后，SageMaker 将切换旧端点，使其指向新部署的模型，并确保整个流程在后台无缝发生。

In [40]:
session.sagemaker_client.update_endpoint(EndpointName=endpoint_name, EndpointConfigName=linear_endpoint_config_name)

{'EndpointArn': 'arn:aws:sagemaker:us-west-1:270372225889:endpoint/boston-update-endpoint-2020-03-31-11-24-04',
 'ResponseMetadata': {'RequestId': '6439f433-93a6-4c4f-b98b-b65eec76dd80',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '6439f433-93a6-4c4f-b98b-b65eec76dd80',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '110',
   'date': 'Tue, 31 Mar 2020 11:31:13 GMT'},
  'RetryAttempts': 0}}

要了解所发生的情况，在更新流程结束之前，我们可以要求 SageMaker 描述目前正在使用的端点。我们发现正在使用的端点依然具有和之前一样的特性。

In [41]:
pprint(session.sagemaker_client.describe_endpoint(EndpointName=endpoint_name))

{'CreationTime': datetime.datetime(2020, 3, 31, 11, 24, 4, 276000, tzinfo=tzlocal()),
 'EndpointArn': 'arn:aws:sagemaker:us-west-1:270372225889:endpoint/boston-update-endpoint-2020-03-31-11-24-04',
 'EndpointConfigName': 'boston-combined-endpoint-config-2020-03-31-11-24-02',
 'EndpointName': 'boston-update-endpoint-2020-03-31-11-24-04',
 'EndpointStatus': 'Updating',
 'LastModifiedTime': datetime.datetime(2020, 3, 31, 11, 31, 13, 983000, tzinfo=tzlocal()),
 'ProductionVariants': [{'CurrentInstanceCount': 1,
                         'CurrentWeight': 1.0,
                         'DeployedImages': [{'ResolutionTime': datetime.datetime(2020, 3, 31, 11, 24, 5, 971000, tzinfo=tzlocal()),
                                             'ResolvedImage': '632365934929.dkr.ecr.us-west-1.amazonaws.com/linear-learner@sha256:1cc553330fc7ab939e72cc0c0ed4bed61bbb2e7b33b4f838cb0a146d0bb5da9c',
                                             'SpecifiedImage': '632365934929.dkr.ecr.us-west-1.amazonaws.com/li

如果现在等待更新流程完成，然后要求 SageMaker 描述端点，它将返回新端点的特性。

In [42]:
endpoint_dec = session.wait_for_endpoint(endpoint_name)

-------------!

In [43]:
pprint(session.sagemaker_client.describe_endpoint(EndpointName=endpoint_name))

{'CreationTime': datetime.datetime(2020, 3, 31, 11, 24, 4, 276000, tzinfo=tzlocal()),
 'EndpointArn': 'arn:aws:sagemaker:us-west-1:270372225889:endpoint/boston-update-endpoint-2020-03-31-11-24-04',
 'EndpointConfigName': 'boston-linear-endpoint-config-2020-03-31-11-16-51',
 'EndpointName': 'boston-update-endpoint-2020-03-31-11-24-04',
 'EndpointStatus': 'InService',
 'LastModifiedTime': datetime.datetime(2020, 3, 31, 11, 37, 23, 209000, tzinfo=tzlocal()),
 'ProductionVariants': [{'CurrentInstanceCount': 1,
                         'CurrentWeight': 1.0,
                         'DeployedImages': [{'ResolutionTime': datetime.datetime(2020, 3, 31, 11, 31, 15, 848000, tzinfo=tzlocal()),
                                             'ResolvedImage': '632365934929.dkr.ecr.us-west-1.amazonaws.com/linear-learner@sha256:1cc553330fc7ab939e72cc0c0ed4bed61bbb2e7b33b4f838cb0a146d0bb5da9c',
                                             'SpecifiedImage': '632365934929.dkr.ecr.us-west-1.amazonaws.com/li

## 关闭端点

使用完毕后，我们需要关闭端点。

In [44]:
session.sagemaker_client.delete_endpoint(EndpointName = endpoint_name)

{'ResponseMetadata': {'RequestId': 'd28bbe65-fbf9-4042-a1f3-72417ef0d487',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd28bbe65-fbf9-4042-a1f3-72417ef0d487',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Tue, 31 Mar 2020 11:37:58 GMT'},
  'RetryAttempts': 0}}

## 可选步骤：清理数据

SageMaker 上的默认 notebook 实例没有太多的可用磁盘空间。当你继续完成和执行 notebook 时，最终会耗尽磁盘空间，导致难以诊断的错误。完全使用完 notebook 后，建议删除创建的文件。你可以从终端或 notebook hub 删除文件。以下单元格中包含了从 notebook 内清理文件的命令。

In [45]:
# First we will remove all of the files contained in the data_dir directory
!rm $data_dir/*

# And then we delete the directory itself
!rmdir $data_dir